In [ ]:
import sqlite3
import zipfile

import pandas as pd

## Conhecer a base de dados dos acórdãos do TCU

A base de dados dos acórdãos do TCU foi disponibilizada para download público na plataforma Kaggle. Os dados dizem respeito aos acórdãos proferidos pelo TCU entre os anos de 1992 até 30/08/2019.
Acesse o endereço <https://www.kaggle.com/ferraz/acordaos-tcu> e baixe o arquivo no link Download (4GB) e salve na pasta '/dados'.
github: <https://github.com/netoferraz/acordaos-tcu>

In [ ]:
with zipfile.ZipFile('dados/330881_660826_bundle_archive.zip', 'r') as zip_ref:
    zip_ref.extractall()

In [ ]:
# Cria conexão com a base de datos sqlite
conn = sqlite3.connect('dados/tcu-acordaos.db')

In [ ]:
# Cria o dataframe df_acordaos
df_acordaos = pd.read_sql_query('SELECT * from acordaos', conn)

In [ ]:
# Salva o arquivo em um csv compactado
df_acordaos.to_csv('dados/df_acordaos.csv', sep='|', index=False, compression='gzip')

In [ ]:
# Lê o arquivo de um csv compactado
df_acordaos = pd.read_csv('dados/df_acordaos.csv', sep='|', compression='gzip', low_memory=False)

In [ ]:
print(df_acordaos.shape)

# Mostra as colunas da base
print(df_acordaos.columns)

In [ ]:
# Remoção dos registros do tipo sigiloso, pois os mesmos não possuem dados disponíveis
df_acordaos = df_acordaos[df_acordaos.tipo_processo != 'SIGILOSO']

In [ ]:
print(df_acordaos.shape)

# Mostra as colunas da base
print(df_acordaos.columns)

In [ ]:
def update_dataframe(df):
    for index, row in df.iterrows():
        if row.acordao.notnull():
            df.at[index, 'tipo'] = 'acordao'
            df.at[index, 'texto'] = df.iloc[index].acordao
        elif row.relatorio.notnull():
            df.at[index, 'tipo'] = 'relatorio'
            df.at[index, 'texto'] = df.iloc[index].relatorio
        elif row.voto.notnull():
            df.at[index, 'tipo'] = 'voto'
            df.at[index, 'texto'] = df.iloc[index].voto
        else:
            print('Nada foi feito na linha: ', index)

In [ ]:
# Retira colunas que não interessam para o trabalho
df_acordaos = df_acordaos.drop(['representante_mp', 'assunto', 'unidade_tecnica', 'repr_legal', 'quorum'], axis=1)

In [ ]:
df_acordaos = df_acordaos[df_acordaos.processo.notnull()]

In [ ]:
# Cria coluna "decisao" contendo a união dos textos das colunas "relatorio", "voto" e "acordao"
df_acordaos['decisao'] = df_acordaos.relatorio + ' ' + df_acordaos.voto + ' ' + df_acordaos.acordao

In [ ]:
print(df_acordaos.shape)

In [ ]:
# Salva o arquivo em um csv compactado
df_acordaos.to_csv('dados/df_acordaos.csv', sep='|', index=False, compression='gzip')